# TidyModels and mlr3

## Aufbau

- Kurz was zu den Paketen und den Entwickler:innen (also TidyModels z.B. Vorgänger caret etc.; mlr3 Vorgänger mlr etc.)
- Installation der Pakete (wie viel ist zu installieren, das ist ja im tidyverse immer so ein Problem, meist sehr viele Pakete)
- Vorstellen eines Cases, ich würde hier tatsächlich den Lending Club Datensatz vorschlagen, da der so um die 30.000 Fälle hat, aber nur wenige Variablen. Das ist dann ein Regressionsbeispiel
- Einfach mal zeigen, was da wie an Data Preparation gemacht werden kann (der Datensatz ist fertig aufbereitet, notwendig ist nur die Dummy-Bildung der kategorialen Variablen, da ist das Paket FastDummies wirklich genial, das könnten wir nutzen).
    - Dummy Preparation
    - Standardisieren (evtl.)
    - Log der Zielvariablen (Diskussion)
    - Split in Training und Test
- Die Schritte können wir dann in eine Pipeline gießen (deshalb sind die Schritte interessant): in TidyModel heißt das nicht Pipeline, sondern Workflow
- Dann einige Algorithmen -> es spielt nicht so eine große Rolle, wie viel wir von den Algorithmen wissen, da die Optimierung ja eh über die Hyperparametersuche passiert.
- Und schließlich alles in das Hyperparameter-Tuning
- (Natürlich noch Evaluation und Prediction)

## Package Handling

Load packages `mlr3` and `tidymodels`

In [2]:
LoadAndOrInstallAPackage <- function(PackageName) {
  # check if input is a character
  if (!is.character(PackageName)) {
    stop("Please provide the Package Name as a character!")
  }
  # check if package is already installed and if not, install it
  if (!require(PackageName, character.only = T)) install.packages(PackageName)
  # Load the package if already installed
  invisible(library(PackageName, character.only = T))
}

In [13]:
packages <- c("mlr3", "tidymodels", "skimr")
sapply(packages, function(package) LoadAndOrInstallAPackage(package))

Lade n"otiges Paket: skimr

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
"es gibt kein Paket namens 'skimr'"



Die heruntergeladenen Bin"arpakete sind in 
	/var/folders/2c/by9dxpt93r57m8h8pyjc00tm0000gn/T//RtmpeBa6JI/downloaded_packages



Attache Paket: 'skimr'


Das folgende Objekt ist maskiert 'package:mlr3':

    partition




$mlr3
 [1] "yardstick"    "workflowsets" "workflows"    "tune"         "tidyr"       
 [6] "tibble"       "rsample"      "recipes"      "purrr"        "parsnip"     
[11] "modeldata"    "infer"        "ggplot2"      "dplyr"        "dials"       
[16] "scales"       "broom"        "tidymodels"   "mlr3"         "stats"       
[21] "graphics"     "grDevices"    "utils"        "datasets"     "methods"     
[26] "base"        

$tidymodels
 [1] "yardstick"    "workflowsets" "workflows"    "tune"         "tidyr"       
 [6] "tibble"       "rsample"      "recipes"      "purrr"        "parsnip"     
[11] "modeldata"    "infer"        "ggplot2"      "dplyr"        "dials"       
[16] "scales"       "broom"        "tidymodels"   "mlr3"         "stats"       
[21] "graphics"     "grDevices"    "utils"        "datasets"     "methods"     
[26] "base"        

$skimr
 [1] "skimr"        "yardstick"    "workflowsets" "workflows"    "tune"        
 [6] "tidyr"        "tibble"       "rsample"      "recipes"      "purrr"       
[11] "parsnip"      "modeldata"    "infer"        "ggplot2"      "dplyr"       
[16] "dials"        "scales"       "broom"        "tidymodels"   "mlr3"        
[21] "stats"        "graphics"     "grDevices"    "utils"        "datasets"    
[26] "methods"      "base"

## Data IO

In [7]:
URL <- "https://raw.githubusercontent.com/gmoeser/wiebaden_R_User_Group/master/LoanStats.csv"
LoanData <- read.csv(URL)
colnames(LoanData)[1] <- "int_rate"

In [8]:
head(LoanData)

,int_rate,loan_amnt,term,grade,home_ownership,annual_inc,purpose
,<dbl>,<int>,<int>,<chr>,<chr>,<dbl>,<chr>
1,10.65,5000,36,B,RENT,24000,credit_card
2,15.27,2500,60,C,RENT,30000,car
3,15.96,2400,36,C,RENT,12252,small_business
4,13.49,10000,36,C,RENT,49200,other
5,12.69,3000,60,B,RENT,80000,other
6,7.90,5000,36,A,RENT,36000,wedding


In [17]:
skimr::skim(LoanData)

-- Data Summary ------------------------
                           Values  
Name                       LoanData
Number of rows             39786   
Number of columns          7       
_______________________            
Column type frequency:             
  character                3       
  numeric                  4       
________________________           
Group variables            None    

-- Variable type: character ----------------------------------------------------
  skim_variable  n_missing complete_rate min max empty n_unique whitespace
1 grade                  0             1   1   1     0        7          0
2 home_ownership         0             1   3   8     0        5          0
3 purpose                0             1   3  18     0       14          0

-- Variable type: numeric ------------------------------------------------------
  skim_variable n_missing complete_rate    mean       sd      p0      p25
1 int_rate              0             1    12.0     3.73    5

Warning message in is.null(text_repr) || nchar(text_repr) == 0L:
"'length(x) = 17 > 1' in coercion to 'logical(1)'"


,skim_type,skim_variable,n_missing,complete_rate,character.min,character.max,character.empty,character.n_unique,character.whitespace,numeric.mean,numeric.sd,numeric.p0,numeric.p25,numeric.p50,numeric.p75,numeric.p100,numeric.hist
,<chr>,<chr>,<int>,<dbl>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,character,grade,0,1,1,1,0,7,0,NA,NA,NA,NA,NA,NA,NA,NA
2,character,home_ownership,0,1,3,8,0,5,0,NA,NA,NA,NA,NA,NA,NA,NA
3,character,purpose,0,1,3,18,0,14,0,NA,NA,NA,NA,NA,NA,NA,NA
4,numeric,int_rate,0,1,NA,NA,NA,NA,NA,12.02787,3.727466,5.42,9.25,11.86,14.59,2.459e+01,<U+2586><U+2587><U+2586><U+2582><U+2581>
5,numeric,loan_amnt,0,1,NA,NA,NA,NA,NA,11231.36028,7464.542832,500.00,5500.00,10000.00,15000.00,3.500e+04,<U+2587><U+2587><U+2583><U+2582><U+2581>
6,numeric,term,0,1,NA,NA,NA,NA,NA,42.44850,10.638782,36.00,36.00,36.00,60.00,6.000e+01,<U+2587><U+2581><U+2581><U+2581><U+2583>
7,numeric,annual_inc,0,1,NA,NA,NA,NA,NA,68979.06676,63762.634520,4000.00,40500.00,59000.00,82342.50,6.000e+06,<U+2587><U+2581><U+2581><U+2581><U+2581>
